# utils & framework

In [1]:
from collections.abc import Iterable

from IPython.core.display import Markdown
from sympy import *

cond_id = 0
var_id = 0

conditions = []
variables = {}

def save_as_condition(condition):
    global cond_id, conditions
    cached_condition_id = cond_id
    conditions.append(condition)
    cond_id += 1
    return cached_condition_id

def substitute_values(expression):
    while True:
        left= False
        for (key, value) in variables.items():
            before = str(expression)
            expression = expression.subs(key, value)
            expression = simplify(expression)
            if str(expression) != before:
                left = True

        if not left:
            break

    return expression


def save_as_variable(symbol, value):
    global variables
    variables[symbol] = substitute_values(value)

def get_conditions(ids) -> tuple:
    return tuple(map(lambda i: conditions[i], ids))


def beautiful_print(expression, message="", tag=None):
    # escape the message for LaTeX and append the expression
    inline = not message.endswith('$')
    tag_str = f'\\tag{{{tag}}}' if tag is not None else ""
    if inline:
        full_latex = message + '$' + latex(expression) + '$'
    else:
        full_latex = message + '$' + latex(expression) + tag_str + '$$'
    display(Markdown(full_latex))


def beautiful_solve(cond, variable):
    cond = [cond] if not isinstance(cond, Iterable) else cond
    variable = [variable] if not isinstance(variable, Iterable) else variable
    solution = solve(get_conditions(cond), tuple(variable), dict=True)[0]
    conclusions = []
    display(Markdown(f"${','.join(f"\\left ( {i} \\right ) " for i in cond)} \\Longrightarrow$"))
    for v in variable:
        conclusion = solution[v]
        beautiful_print(Eq(v, conclusion), "$")
        save_as_variable(v, conclusion)
        conclusions.append(conclusion)
    return tuple(conclusions)


def beautiful_condition(expression, message="$"):
    beautiful_print(expression(False) if callable(expression) else expression, message, cond_id)
    return save_as_condition(expression(True) if callable(expression) else expression)

def beautiful_compute(symbol, expression,message="$",tag:bool=False):
    beautiful_print(Eq(symbol,expression(False) if callable(expression) else expression), message,cond_id if tag else None)
    save_as_variable(symbol, expression)




# Define variables

In [2]:
t = symbols('t')
varphi = Function('varphi')(t)
rho = Function('rho')(t)
V_a = symbols('V_a')
a = symbols('a')
b = symbols('b')


In [3]:
cond1 = beautiful_condition(lambda e: Eq(V_a, diff(a * tan(varphi), t, evaluate=e)), "by geometry:$")

by geometry:$$V_{a} = \frac{\partial}{\partial t} a \tan{\left(\varphi{\left(t \right)} \right)}\tag{0}$$

In [4]:
varphi_d1 = diff(varphi, t, 1)
varphi_d2 = diff(varphi, t, 2)
beautiful_print(Eq(diff(V_a, t, evaluate=False), 0), "$V_a$ as const :$")
cond2 = beautiful_condition(lambda e: Eq(0, diff(a * tan(varphi), t, 2, evaluate=e)), "so as to:$")
_=beautiful_solve([cond1, cond2], [varphi_d1, varphi_d2])

$V_a$ as const :$$\frac{d}{d t} V_{a} = 0$$

so as to:$$0 = \frac{\partial^{2}}{\partial t^{2}} a \tan{\left(\varphi{\left(t \right)} \right)}\tag{1}$$

$\left ( 0 \right ) ,\left ( 1 \right )  \Longrightarrow$

$$\frac{d}{d t} \varphi{\left(t \right)} = \frac{V_{a}}{a \left(\tan^{2}{\left(\varphi{\left(t \right)} \right)} + 1\right)}$$

$$\frac{d^{2}}{d t^{2}} \varphi{\left(t \right)} = - \frac{2 V_{a}^{2} \tan{\left(\varphi{\left(t \right)} \right)}}{a^{2} \left(\tan^{2}{\left(\varphi{\left(t \right)} \right)} + 1\right)^{2}}$$

In [5]:
beautiful_compute(rho,(a * sec(varphi) - b),"by geometry: $",True)

by geometry: $$\rho{\left(t \right)} = a \sec{\left(\varphi{\left(t \right)} \right)} - b\tag{2}$$

In [6]:
velocity = Matrix([diff(rho,t), rho * diff(varphi,t)])
beautiful_print(velocity, r"velocity vector in polar coordinates:$")
velocity = substitute_values(velocity)
beautiful_print(velocity, r"and inlined:$")
beautiful_print(simplify(velocity.norm()), r"speed magnitude:$",tag="answer")


velocity vector in polar coordinates:$$\left[\begin{matrix}\frac{d}{d t} \rho{\left(t \right)}\\\rho{\left(t \right)} \frac{d}{d t} \varphi{\left(t \right)}\end{matrix}\right]$$

and inlined:$$\left[\begin{matrix}V_{a} \sin{\left(\varphi{\left(t \right)} \right)}\\\frac{V_{a} \left(a - b \cos{\left(\varphi{\left(t \right)} \right)}\right) \cos{\left(\varphi{\left(t \right)} \right)}}{a}\end{matrix}\right]$$

speed magnitude:$$\sqrt{\left|{V_{a} \sin{\left(\varphi{\left(t \right)} \right)}}\right|^{2} + \left|{\frac{V_{a} \left(a - b \cos{\left(\varphi{\left(t \right)} \right)}\right) \cos{\left(\varphi{\left(t \right)} \right)}}{a}}\right|^{2}}\tag{answer}$$

In [7]:
acceleration = Matrix([(diff(rho, t, 2) - rho * diff(varphi, t) ** 2),
                       (rho * diff(varphi, t, 2) + 2 * diff(rho, t) * diff(varphi, t))])
beautiful_print(acceleration, r"acceleration vector in polar coordinates:$")
acceleration = substitute_values(acceleration)
beautiful_print(acceleration, r"and inlined:$")
beautiful_print(simplify(acceleration.norm()), r"acceleration magnitude:$",tag="answer")


acceleration vector in polar coordinates:$$\left[\begin{matrix}- \rho{\left(t \right)} \left(\frac{d}{d t} \varphi{\left(t \right)}\right)^{2} + \frac{d^{2}}{d t^{2}} \rho{\left(t \right)}\\\rho{\left(t \right)} \frac{d^{2}}{d t^{2}} \varphi{\left(t \right)} + 2 \frac{d}{d t} \rho{\left(t \right)} \frac{d}{d t} \varphi{\left(t \right)}\end{matrix}\right]$$

and inlined:$$\left[\begin{matrix}\frac{V_{a}^{2} b \cos^{4}{\left(\varphi{\left(t \right)} \right)}}{a^{2}}\\\frac{2 V_{a}^{2} b \sin{\left(\varphi{\left(t \right)} \right)} \cos^{3}{\left(\varphi{\left(t \right)} \right)}}{a^{2}}\end{matrix}\right]$$

acceleration magnitude:$$\sqrt{\left|{\frac{V_{a}^{2} b \cos^{4}{\left(\varphi{\left(t \right)} \right)}}{a^{2}}}\right|^{2} + 4 \left|{\frac{V_{a}^{2} b \sin{\left(\varphi{\left(t \right)} \right)} \cos^{3}{\left(\varphi{\left(t \right)} \right)}}{a^{2}}}\right|^{2}}\tag{answer}$$